In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# QUICK-PDE: Eine Qiskit Function von ColibriTD
> **Note:** Qiskit Functions sind eine experimentelle Funktion, die Benutzern des IBM Quantum&reg; Premium Plans, Flex Plans und On-Prem (über IBM Quantum Platform API) Plans zur Verfügung steht. Sie befinden sich im Preview-Release-Status und können sich ändern.
## Überblick
Der hier vorgestellte Solver für partielle Differentialgleichungen (PDE) ist Teil unserer Quantum Innovative Computing Kit (QUICK)-Plattform (QUICK-PDE) und wird als Qiskit Function bereitgestellt. Mit der QUICK-PDE-Funktion können Sie domänenspezifische partielle Differentialgleichungen auf IBM Quantum QPUs lösen. Diese Funktion basiert auf dem in [ColibriTDs H-DES-Beschreibungspapier](https://arxiv.org/abs/2410.01130) beschriebenen Algorithmus. Dieser Algorithmus kann komplexe Multiphysik-Probleme lösen, beginnend mit Computational Fluid Dynamics (CFD) und Materials Deformation (MD), wobei weitere Anwendungsfälle in Kürze folgen.

Um die Differentialgleichungen anzugehen, werden die Testlösungen als Linearkombinationen orthogonaler Funktionen (typischerweise Chebyshev-Polynome, und genauer $2^n$ davon, wobei $n$ die Anzahl der Qubits ist, die Ihre Funktion kodieren) kodiert, parametrisiert durch die Winkel einer Variable Quantum Circuit (VQC). Der Ansatz erzeugt einen Zustand, der die Funktion kodiert, die durch Observablen ausgewertet wird, deren Kombinationen die Auswertung der Funktion an allen Punkten ermöglichen. Sie können dann die Verlustfunktion auswerten, in der die Differentialgleichungen kodiert sind, und die Winkel in einer hybriden Schleife feinabstimmen, wie im Folgenden gezeigt. Die Testlösungen kommen den tatsächlichen Lösungen schrittweise näher, bis Sie ein zufriedenstellendes Ergebnis erreichen.

![Workflow der QUICK-PDE-Funktion](../docs/images/guides/colibritd-equation-solver/diagram.svg)

Zusätzlich zu dieser hybriden Schleife können Sie auch verschiedene Optimierer verketten. Dies ist nützlich, wenn Sie einen globalen Optimierer verwenden möchten, um einen guten Satz von Winkeln zu finden, und dann einen feineren Optimierer, um einem Gradienten zum besten Satz benachbarter Winkel zu folgen. Im Fall der Computational Fluid Dynamics (CFD) liefert die Standard-Optimierungssequenz die besten Ergebnisse - aber im Fall der Material Deformation (MD) können Sie, während der Standard gute Ergebnisse liefert, ihn für problemspezifische Vorteile weiter konfigurieren.

Beachten Sie, dass wir für jede Variable der Funktion die Anzahl der Qubits angeben (mit der Sie experimentieren können). Durch das Stapeln von 10 identischen Schaltungen und die Auswertung der 10 identischen Observablen auf verschiedenen Qubits durch eine große Schaltung können Sie innerhalb des CMA-Optimierungsprozesses Rauschminderung durchführen, wobei Sie auf die Noise-Learner-Methode vertrauen und die Anzahl der benötigten Shots erheblich reduzieren.
## Eingabe/Ausgabe
### Computational Fluid Dynamics

Die inviszide Burgers-Gleichung modelliert strömende nicht-viskose Flüssigkeiten wie folgt:

$$\frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} = 0,$$

$u$ stellt das Fluidgeschwindigkeitsfeld dar. Dieser Anwendungsfall hat eine zeitliche Randbedingung: Sie können die Anfangsbedingung auswählen und dann das System relaxieren lassen. Derzeit sind die einzigen akzeptierten Anfangsbedingungen lineare Funktionen: $ax + b$.

Die Argumente für CFDs Differentialgleichungen befinden sich auf einem festen Gitter wie folgt:

- $t$ liegt zwischen 0 und 0,95 mit 30 Abtastpunkten. $x$ liegt zwischen 0 und 0,95 mit einer Schrittweite von 0,2375.

### Material Deformation

Dieser Anwendungsfall konzentriert sich auf hypoelastische Verformung mit dem eindimensionalen Zugversuch, bei dem ein im Raum fixierter Stab an seinem anderen Ende gezogen wird. Wir beschreiben das Problem wie folgt:

$$u' - \frac{\sigma}{3K} - \frac{2}{\sqrt{3}}\epsilon_0\left(\frac{\sigma'}{\sigma_0\sqrt{3}}\right)^n = 0$$

$$\sigma' - b = 0,$$

$K$ stellt den Kompressionsmodul des gedehnten Materials dar, $n$ den Exponenten eines Potenzgesetzes, $b$ die Kraft pro Masseneinheit, $\epsilon_0$ die Proportionalitätsspannungsgrenze, $\sigma_0$ die Proportionalitätsdehnungsgrenze, $u$ die Spannungsfunktion und $\sigma$ die Dehnungsfunktion.

Der betrachtete Stab hat eine Einheitslänge. Dieser Anwendungsfall hat eine Randbedingung für Oberflächenspannung $t$, oder die Menge an Arbeit, die benötigt wird, um den Stab zu dehnen.

Die Argumente für MDs Differentialgleichungen befinden sich auf einem festen Gitter wie folgt:

- $x$ liegt zwischen 0 und 1 mit einer Schrittweite von 0,04.
### Eingabe
Um die QUICK-PDE Qiskit Function auszuführen, können Sie die folgenden Parameter anpassen:

| Name              | Typ                                                  | Beschreibung                                                                                                                                                                                                                                                                                  | Anwendungsfallspezifisch | Beispiel                 |
| ----------------- | ---------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------------ | ------------------------ |
| use_case          | `Literal["MD", "CFD"]`                               | Umschalter zur Auswahl des zu lösenden Systems von Differentialgleichungen                                                                                                                                                                                                                    | Nein                     | `"CFD"`                  |
| parameters        | `dict[str, Any]`                                     | Parameter der Differentialgleichung (weitere Details siehe nächste Tabelle)                                                                                                                                                                                                                   | Nein                     | `{"a": 1.0, "b": 1.0}`   |
| nb_qubits         | `Optional[dict[str, dict[str, int]]]`                | Anzahl der Qubits pro Funktion und pro Variable. Optimierte Werte werden von der Funktion ausgewählt, aber wenn Sie versuchen möchten, eine bessere Kombination zu finden, können Sie die Standardwerte überschreiben                                                                        | Nein                     | `{"u": {"x": 1, "t":3}}` |
| depth             | `Optional[dict[str, int]]`                           | Tiefe des Ansatzes pro Funktion. Optimierte Werte werden von der Funktion ausgewählt, aber wenn Sie versuchen möchten, eine bessere Kombination zu finden, können Sie die Standardwerte überschreiben                                                                                        | Nein                     | `{"u": 4}`               |
| optimizer         | `Optional[list[str]]`                                | Zu verwendende Optimierer, entweder "CMAES" aus der [`cma` Python-Bibliothek](https://github.com/CMA-ES/pycma) oder einer der [scipy-Optimierer](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)                                                          | MD                       | `"SLSQP"`                |
| shots             | `Optional[list[int]]`                                | Anzahl der Shots zur Ausführung jeder Schaltung. Da mehrere Optimierungsschritte erforderlich sind, muss die Länge der Liste gleich der Anzahl der verwendeten Optimierer sein (4 im Fall von CFD). Standard ist `[50_000] * nb_optimizers` für MD und `[5_00, 2_000, 5_000, 10_000]` für CFD | Nein                     | `[15_000, 30_000]`       |
| optimizer_options | `Optional[dict[str, Any]]`                           | Optionen zur Übergabe an den Optimierer. Die Details dieser Eingabe hängen vom verwendeten Optimierer ab; für Einzelheiten siehe die Dokumentation des verwendeten Optimierers                                                                                                                | MD                       | `{"maxiter": 50 }`       |
| initialization    | `Optional[Literal["RANDOM", "PHYSICALLY_INFORMED"]]` | Ob mit zufälligen Winkeln oder intelligent gewählten Winkeln begonnen werden soll. Beachten Sie, dass intelligent gewählte Winkel möglicherweise nicht in 100% der Fälle funktionieren. Standard ist `"PHYSICALLY_INFORMED"`.                                                                 | Nein                     | `"RANDOM"`               |
| backend_name      | `Optional[str]`                                      | Der zu verwendende Backend-Name.                                                                                                                                                                                                                                                              | Nein                     | `"ibm_torino"`           |
| mode              | `Optional[Literal["job", "session", "batch"]]`       | Der zu verwendende Ausführungsmodus. Standard ist `"job"`.                                                                                                                                                                                                                                    | Nein                     | `"job"`                  |

Die Parameter der Differentialgleichung (physikalische Parameter und Randbedingung) sollten dem angegebenen Format folgen:

| Anwendungsfall | Schlüssel   | Wertetyp | Beschreibung                            | Beispiel |
| -------------- | ----------- | -------- | --------------------------------------- | -------- |
| CFD            | `a`         | `float`  | Koeffizient der Anfangswerte von $u$    | `1.0`    |
| CFD            | `b`         | `float`  | Versatz der Anfangswerte von $u$        | `1.0`    |
| MD             | `t`         | `float`  | Oberflächenspannung                     | `12.0`   |
| MD             | `K`         | `float`  | Kompressionsmodul                       | `100.0`  |
| MD             | `n`         | `int`    | Potenzgesetz                            | `4.0`    |
| MD             | `b`         | `float`  | Kraft pro Masseneinheit                 | `10.0`   |
| MD             | `epsilon_0` | `float`  | Proportionalitätsspannungsgrenze        | `0.1`    |
| MD             | `sigma_0`   | `float`  | Proportionalitätsdehnungsgrenze         | `5.0`    |
### Ausgabe
Die Ausgabe ist ein Wörterbuch mit der Liste der Abtastpunkte sowie den Werten der Funktionen an jedem dieser Punkte:

In [ ]:
from numpy import array

In [ ]:
solution = {
    "functions": {
        "u": array(
            [
                [0.01, 0.1, 1],
                [0.02, 0.2, 2],
                [0.03, 0.3, 3],
                [0.04, 0.4, 4],
            ]
        ),
    },
    "samples": {
        "t": array([0.1, 0.2, 0.3, 0.4]),
        "x": array([0.5, 0.6, 0.7]),
    },
}

Die Form eines Lösungsarrays hängt von den Abtastpunkten der Variablen ab:

In [ ]:
assert len(solution["functions"]["u"].shape) == len(solution["samples"])
for col_size, samples in zip(
    solution["functions"]["u"].shape, solution["samples"].values()
):
    assert col_size == len(samples)

Die Zuordnung zwischen den Abtastpunkten der Funktionsvariablen und der Dimension des Lösungsarrays erfolgt in alphanumerischer Reihenfolge des Variablennamens. Wenn die Variablen beispielsweise `"t"` und `"x"` sind, repräsentiert eine Zeile von `solution["functions"]["u"]` die Werte der Lösung für ein festes `"t"`, und eine Spalte von `solution["functions"]["u"]` repräsentiert die Werte der Lösung für ein festes `"x"`.

Das Folgende ist ein Beispiel dafür, wie Sie den Wert der Funktion für einen bestimmten Satz von Koordinaten erhalten:

In [ ]:
# u(t=0.2, x=0.7) == 2
assert solution["samples"]["t"][1] == 0.2
assert solution["samples"]["x"][2] == 0.7
assert solution["functions"]["u"][1, 2] == 2

## Benchmarks


Die folgende Tabelle zeigt Statistiken zu verschiedenen Ausführungen unserer Funktion.

| Beispiel                              | Anzahl der Qubits | Initialisierung       | Fehler    | Gesamtzeit (min) | Runtime-Nutzung (min) |
| ------------------------------------- | ----------------- | --------------------- | --------- | ---------------- | --------------------- |
| Inviszide Burgers-Gleichung           | 50                | `PHYSICALLY_INFORMED` | $10^{-2}$ | 66               | 25                    |
| Hypoelastischer 1D-Zugversuch         | 18                | `RANDOM`              | $10^{-2}$ | 123              | 100                   |

## Erste Schritte

Füllen Sie das [Formular aus, um Zugang zur QUICK-PDE-Funktion anzufordern.](https://forms.cloud.microsoft/e/3Wi9cbjQPK) Anschließend, vorausgesetzt Sie haben bereits [Ihr Konto gespeichert](/guides/functions#install-qiskit-functions-catalog-client) in Ihrer lokalen Umgebung, wählen Sie die Funktion wie folgt aus:

In [1]:
from qiskit_ibm_catalog import QiskitFunctionsCatalog

catalog = QiskitFunctionsCatalog(channel="ibm_quantum_platform")

quick = catalog.load("colibritd/quick-pde")

## Beispiele

Um loszulegen, probieren Sie eines der folgenden Beispiele:

### Computational Fluid Dynamics (CFD)


Wenn Anfangsbedingungen auf $u(0,x) = x$ gesetzt sind, sind die Ergebnisse wie folgt:

In [ ]:
# launch the simulation with initial conditions u(0,x) = a*x + b
job = quick.run(use_case="cfd", physical_parameters={"a": 1.0, "b": 0.0})

Überprüfen Sie den [Status](/guides/functions#check-job-status) Ihrer Qiskit Function-Workload oder geben Sie [Ergebnisse](/guides/functions#retrieve-results) wie folgt zurück:

In [ ]:
print(job.status())
solution = job.result()

'QUEUED'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

_ = plt.figure()
ax = plt.axes(projection="3d")

# plot the solution using the 3d plotting capabilities of pyplot
t, x = np.meshgrid(solution["samples"]["t"], solution["samples"]["x"])
ax.plot_surface(
    t,
    x,
    solution["functions"]["u"],
    edgecolor="royalblue",
    lw=0.25,
    rstride=26,
    cstride=26,
    alpha=0.3,
)
ax.scatter(t, x, solution["functions"]["u"], marker=".")
ax.set(xlabel="t", ylabel="x", zlabel="u(t,x)")

plt.show()

<Image src="../docs/images/guides/colibritd-pde/extracted-outputs/c42aba9b-0.avif" alt="Output of the previous code cell" />

### Material Deformation

The material deformation use case requires the physical parameters of your material and the applied force, as follows:

In [ ]:
import matplotlib.pyplot as plt

# select the properties of your material
job = quick.run(
    use_case="md",
    physical_parameters={
        "t": 12.0,
        "K": 100.0,
        "n": 4.0,
        "b": 10.0,
        "epsilon_0": 0.1,
        "sigma_0": 5.0,
    },
)

# plot the result
solution = job.result()

_ = plt.figure()
stress_plot = plt.subplot(211)
plt.plot(solution["samples"]["x"], solution["functions"]["u"])
strain_plot = plt.subplot(212)
plt.plot(solution["samples"]["x"], solution["functions"]["sigma"])

plt.show()

<Image src="../docs/images/guides/colibritd-pde/extracted-outputs/a568e325-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/colibritd-pde/extracted-outputs/c42aba9b-0.avif)

### Material Deformation

Der Material Deformation-Anwendungsfall erfordert die physikalischen Parameter Ihres Materials und die angewandte Kraft wie folgt:

In [ ]:
job = quick.run(use_case="mdf", physical_params={})

print(job.error_message())


# or write a wrapper around it for a more human readable version
def pprint_error(job):
    print("".join(eval(job.error_message())["error"]))


print("___")
pprint_error(job)

{"error": ["qiskit.exceptions.QiskitError: 'Unknown argument \"physical_params\", did you make a typo? -- https://docs.quantum.ibm.com/errors#1804'\n"]}
___
qiskit.exceptions.QiskitError: 'Unknown argument "physical_params", did you make a typo? -- https://docs.quantum.ibm.com/errors#1804'



![Output of the previous code cell](../docs/images/guides/colibritd-pde/extracted-outputs/a568e325-0.avif)

## Fehlermeldungen abrufen

Wenn der Status Ihrer Workload `ERROR` ist, verwenden Sie `job.error_message()`, um die Fehlermeldung zum Debuggen abzurufen, wie folgt: